In [ ]:
#  Copyright (c) 2023 Snowflake Computing Inc. All rights reserved.

# Import python packages & establish session
import streamlit as st # Import python packages
import pandas as pd
from PyPDF2 import PdfFileReader
from snowflake.snowpark.files import SnowflakeFile
from io import BytesIO
from snowflake.snowpark.types import StringType, StructField, StructType
from langchain.text_splitter import RecursiveCharacterTextSplitter

from snowflake.snowpark.context import get_active_session
session = get_active_session()

print('Session Created!')
#test23



RAG Made Easy w/ Snowflake Cortex
========

Creating an end-to-end Retrieval Augmented Generation process (or RAG) directly in Snowflake.
1) Extract full text from PDF files using Snowpark.
2) Chunk those documents using Langchain in Snowpark.
3) Use Cortex to create embeddings of those chunks.
4) Use Vector Similarity to show the most similar chunk when prompting an LLM.

In [ ]:
-- Optional set up: Place your MD files in a stage for extraction
--ls @INTERNALSTORAGE;

select * from directory(@internalstorage);

In [ ]:
st.image( "https://raw.githubusercontent.com/NickAkincilar/DemoContent/master/Extract_Text.png" )

In [ ]:
#Create a Snowpark based function to extract text from PDFs
__name__ = '__main__'
def readpdf(file_path):
    whole_text = ""
    with SnowflakeFile.open(file_path, 'rb') as file:
        f = BytesIO(file.readall())
        pdf_reader = PdfFileReader(f)
        whole_text = ""
        for page in pdf_reader.pages:
            whole_text += page.extract_text()
    return whole_text

In [ ]:
#Register the UDF. 
#Optional : Convert the cell to markdown to prevent rerunning later.
session.udf.register(
    func = readpdf
  , return_type = StringType()
  , input_types = [StringType()]
  , is_permanent = True
  , name = 'SNOWPARK_PDF'
  , replace = True
  , packages=['snowflake-snowpark-python','pypdf2']
  , stage_location = 'DOC_AI_DB.LLM_DEMO.MY_UDF_STAGE'
)

In [ ]:
st.image( "https://raw.githubusercontent.com/NickAkincilar/DemoContent/master/Extract_Text.png" )

In [ ]:
CREATE OR REPLACE TABLE RAW_TEXT AS
SELECT
    relative_path
    , file_url
    , snowpark_pdf(build_scoped_file_url(@internalstorage, relative_path)) as raw_text
from directory(@internalstorage);

In [ ]:
select * from raw_text;

In [ ]:
st.image( "https://raw.githubusercontent.com/NickAkincilar/DemoContent/master/Extract_web.png" )

In [ ]:
insert into RAW_TEXT 
select 
'NBA', 
'privacy-policy', 
deletethis.public.webscrape_via_python('https://www.nba.com/privacy-policy')


In [ ]:
SELECT
*
FROM RAW_TEXT
where relative_path ilike '%nba%'

In [ ]:
--Optional : This will fail due to tokens exceeding limit, which means we need to chunk!
SELECT
SNOWFLAKE.CORTEX.COMPLETE('llama2-70b-chat','What year Snowflake Inc. was founded?') as Answer

In [ ]:
--Optional : This will fail due to tokens exceeding limit, which means we need to chunk!
SELECT
SNOWFLAKE.CORTEX.COMPLETE('llama2-70b-chat','summarise the following text:' || raw_text) 
FROM
RAW_TEXT
LIMIT 1;

A note on chunking
-----
Chunking is the process of splitting a large body of text into smaller 'chunks' whilst attempting to keep as much relevant information as possible. Make the chunks too small and you run the risk of removing key information that the model requires to answer the question. Too large and it may be harder to retreive the correct body of text from the vector search - or spend tokens excessively.

There are many strategies towards chunking. Eg - pass the most relevant, top n relevant chunks, or pass the most relevent chunk + the chunk either side of that one. Play around and see what works for your use case!


In [ ]:
st.image( "https://raw.githubusercontent.com/NickAkincilar/DemoContent/master/Chunk.text.png" )

In [ ]:
#A class for chunking text and returning a table via UDTF
__name__ = '__main__'
class text_chunker:

    def process(self,text):        
        text_raw=[]
        text_raw.append(text) 
        
        text_splitter = RecursiveCharacterTextSplitter(
            separators = ["\n"], # Define an appropriate separator. New line is good typically!
            chunk_size = 1000, #Adjust this as you see fit
            chunk_overlap  = 50, #This let's text have some form of overlap. Useful for keeping chunks contextual
            length_function = len,
            add_start_index = True #Optional but useful if you'd like to feed the chunk before/after
        )
    
        chunks = text_splitter.create_documents(text_raw)
        df = pd.DataFrame(chunks, columns=['chunks','meta'])
        
        yield from df.itertuples(index=False, name=None)


In [ ]:
#Register the UDTF - set the stage location

schema = StructType([
     StructField("chunk", StringType()),
    StructField("meta", StringType()),
 ])

session.udtf.register( 
    handler = text_chunker,
    output_schema= schema, 
    input_types = [StringType()] , 
    is_permanent = True , 
    name = 'CHUNK_TEXT' , 
    replace = True , 
    packages=['pandas','langchain'], stage_location = 'DOC_AI_DB.LLM_DEMO.MY_UDF_STAGE' )

In [ ]:
--Create the chunked version of the table
CREATE OR REPLACE TABLE CHUNK_TEXT AS
SELECT
        relative_path,
        func.*
    FROM raw_text AS raw,
         TABLE(chunk_text(raw_text)) as func;

In [ ]:
SELECT * FROM CHUNK_TEXT limit 10;

In [ ]:
st.image( "https://raw.githubusercontent.com/NickAkincilar/DemoContent/master/Embed_text4.png" )

In [ ]:
--Convert your chunks to embeddings
CREATE OR REPLACE TABLE VECTOR_STORE AS
SELECT
RELATIVE_PATH as DOC_NAME,
CHUNK AS CHUNK,
snowflake.cortex.embed_text('e5-base-v2', chunk) as chunk_embedding
FROM CHUNK_TEXT;

In [ ]:
st.image( "https://raw.githubusercontent.com/NickAkincilar/DemoContent/master/Search_text.png" )

In [ ]:
--Vector distance allows use to find the most similar chunk to a question
SELECT DOC_NAME, CHUNK from DOC_AI_DB.DOC_AI_SCHEMA.VECTOR_STORE 
            ORDER BY VECTOR_L2_DISTANCE(
            snowflake.cortex.embed_text('e5-base-v2', 'what is results cache?')
            , CHUNK_EMBEDDING) limit 3
        ;

In [ ]:
st.image( "https://raw.githubusercontent.com/NickAkincilar/DemoContent/master/Complete_LLM1.png" )

In [ ]:

--Pass the chunk we need along with the prompt to get a better structured answer from the LLM
SELECT snowflake.cortex.complete(
    'llama2-70b-chat', CONCAT( 
    'Answer the question based on the context. Be concise.','Context: ',
        (
            SELECT chunk FROM DOC_AI_DB.DOC_AI_SCHEMA.VECTOR_STORE
            ORDER BY vector_l2_distance(
            snowflake.cortex.embed_text('e5-base-v2', 
            'how to disable traction control?'
            ), chunk_embedding
            ) LIMIT 1
        ),
        'Question: ', 
        'how to disable traction control?',
        'Answer: '
    )
) as response;

In [ ]:
df = cells.RAG_Query.to_df()
df.show()

In [ ]:

st.image( "https://media.licdn.com/dms/image/D4E12AQFiylBcFOLdmw/article-inline_image-shrink_1500_2232/0/1716123014576?e=1721865600&v=beta&t=jNmSy7km8OJXai4quTdigYofYhY6Y8oJmbXnh2zdOU0" )

In [ ]:

from snowflake.snowpark.context import get_active_session
session = get_active_session() # Get the current credentials

st.title("Ask Your Data Anything :snowflake:")
st.write("""Built using end-to-end RAG in Snowflake with Cortex functions.""")

model = st.selectbox('Select your model:',('snowflake-arctic', 'mistral-large','llama2-70b-chat','mixtral-8x7b','gemma-7b','mistral-7b'))

prompt = st.text_input("Enter prompt", placeholder="How much does the car weigh?", label_visibility="collapsed")

quest_q = f'''
select snowflake.cortex.complete(
    '{model}', 
    concat( 
        'Answer the question based on the context. Be consise','Context: ',
        (
            select chunk from DOC_AI_DB.DOC_AI_SCHEMA.VECTOR_STORE 
            order by vector_l2_distance(
            snowflake.cortex.embed_text('e5-base-v2', 
            '{prompt}'
            ), chunk_embedding
            ) limit 1
        ),
        'Question: ', 
        '{prompt}',
        'Answer: '
    )
) as response;
'''

if prompt:
    df_query = session.sql(quest_q).to_pandas()
    st.write(df_query['RESPONSE'][0])